# Импортирование библиотек

In [1]:
import warnings
warnings.filterwarnings('ignore')

import re

from sklearn.datasets import fetch_20newsgroups

import numpy as np

import matplotlib.pyplot as plt

import collections

import torch
from torch import nn
from torch.nn import functional as F

# Считывание и предобработка текстов

In [2]:
train = fetch_20newsgroups(subset = 'train')
test = fetch_20newsgroups(subset = 'test')

In [3]:
print('Количество обучающих текстов: ', len(train['data']))
print('Количество тестовых текстов: ', len(test['data']))
print()
print(train['data'][0].strip())

print()
print('Метка: ',train['target'][0])

Количество обучающих текстов:  11314
Количество тестовых текстов:  7532

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----

Метка:  7


## Токенизация, удаление слишком коротких слов, удаление стоп-слов, лемматизация.

In [4]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

def tokenizer(text):
    txt = text.lower()
    token_re = re.compile(r"\d+[s]+|[-]?\d+[.,]?\d+|\w+[-]+\w+[-]?\w+|\w+[']?\w+|\w+")
    txt_tokens = token_re.findall(txt)
    tokens = [token for token in txt_tokens if len(token) >= 4]
    
    lemmatizer = WordNetLemmatizer()
    token_num = 0
    for token in tokens:
        lemmatize_token = lemmatizer.lemmatize(token)
        tokens[token_num] = lemmatize_token
        token_num += 1
    
    english_stop_words = set(stopwords.words('english'))
    return [token for token in tokens if not token in english_stop_words]

# Векторизация модели при помощи TF-IDF, классификация логистической регрессией

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

model = Pipeline((('vectorize', TfidfVectorizer(tokenizer = tokenizer, max_df = 0.7)),
                 ('classification', LogisticRegression())))

model.fit(train['data'], train['target'])

Pipeline(memory=None,
         steps=[('vectorize',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.7, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_patte...
                                 tokenizer=<function tokenizer at 0x000001D0EBC6E9D8>,
                                 use_idf=True, vocabulary=None)),
                ('classification',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_

In [6]:
train_predictions = model.predict(train['data'])
print('Точность предсказаний на тренировочной выборке: ', accuracy_score(train['target'], train_predictions))

print()

test_predictions = model.predict(test['data'])
print('Точность предсказаний на тренировочной выборке: ', accuracy_score(test['target'], test_predictions))

Точность предсказаний на тренировочной выборке:  0.9776383242001061

Точность предсказаний на тренировочной выборке:  0.8224907063197026


### *Модель переобучилась, но все равно выдает достаточно неплохой результат на тестовой выборке. Где-то 4 из 5 текстов классификатор правильно классифицирует.*